### Descripción

Contexto de negocio.

El cáncer de próstata es uno de los tipos de cáncer más comunes en hombres. La detección temprana es crucial para mejorar las tasas de supervivencia. La prueba de antígeno prostático específico (PSA) puede ayudar a detectar el cáncer de próstata en etapas tempranas, cuando es más tratable. Sin embargo, el PSA no es específico para el cáncer de próstata y puede estar elevado en otras condiciones como prostatitis o hiperplasia prostática benigna (HPB).

El antígeno prostático específico (PSA) es una proteína producida por células normales y malignas de la glándula prostática. La prueba del PSA mide el nivel de esta proteína en la sangre y es uno de los métodos más utilizados para el tamizaje del cáncer de próstata.

La EPS SaludPorTi, está interesado en priorizar la toma de está prueba, aumentando la demanda y detención temprana del Cáncer de Próstata.

Problema de negocio
La empresa ha decidido contratarlos para que construyan un modelo predictivo que permita estimar la probabilidad de que un usuario entre 48 y 60 años de edad presente resultados anormales de PSA.


Contexto analítico

Se espera que entrene diferentes familias de modelos predictivos de clasificación (SVC con diferentes kernels, Redes Neuronales poco profundas), precedidos por diferentes procesos de transformación (normalizaciones, imputación, feature engineering, dummificación, PCA, selección de features).


La evaluación de la calidad de los flujos de modelos predictivos se debe estimar utilizando la métrica de ROC_AUC.

Expliquen sus ideas, el por qué realizan las acciones, y comenten los resultados obtenidos; se espera mucho más que unos bloques de código.
La toma de decisiones sobre los datos se debe hacer considerando el contexto del problema y de los datos, no se puede ver todo solamente desde los ojos de los datos, sino también considerar el negocio.
Un Científico de Datos debe poder comunicar los puntos importantes de su trabajo en un lenguaje universal para todos los públicos.
Todo esto se considerará en la nota.

### Cita

@misc{fa-ii-2024-ii-flujos-de-modelos-tradicionales,
    author = {Daniel Osorio, JavierDiaz},
    title = {FA II 2024-II: flujos de modelos tradicionales},
    publisher = {Kaggle},
    year = {2024},
    url = {https://kaggle.com/competitions/fa-ii-2024-ii-flujos-de-modelos-tradicionales}
}

### 1. Librerías a básicas utilizar

In [1]:
# Importar librerías

# Manejo de analísis de datos a través de dataframes (data tabular)
import pandas as pd
# Manipulación de arreglos y análisis numérico
import numpy as np
# Visualización de datos
import seaborn as sns
# Visualización de datos
import matplotlib.pyplot as plt
%matplotlib inline
# Librería para el manejo de expresiones regulares
import re

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from bayes_opt import BayesianOptimization


import warnings
warnings.filterwarnings("ignore")


### 2. Lectura de datos

In [ ]:
df_train = pd.read_parquet(r"https://github.com/alfa7g7/Fundamentos-analitica-II/raw/refs/heads/main/UNIDAD%20II/Data/df_train.parquet")
print(df_train.shape)
df_train.head()

In [ ]:
df_test = pd.read_parquet(r"https://github.com/alfa7g7/Fundamentos-analitica-II/raw/refs/heads/main/UNIDAD%20II/Data/df_test.parquet")
print(df_test.shape)
df_test.head()

In [28]:
#pasar a csv
#df.to_csv(r"C:\Users\alfa7\OneDrive\Documentos\ICESI\MAESTRIA CIENCIA DE DATOS\2do semestre\Fundamentos de analitica II\Unidad II\Proyecto PSA\Data\df_train.csv")
#df1.to_csv(r"C:\Users\alfa7\OneDrive\Documentos\ICESI\MAESTRIA CIENCIA DE DATOS\2do semestre\Fundamentos de analitica II\Unidad II\Proyecto PSA\Data\df_test.csv")

### 3. Análisis Explotarorio de Datos

- Limpieza de datos
- Exploración de datos
- Ingeniería de datos

### 3.1. Limpieza de datos

Para realizar una apropiada limpieza de datos se deben entender cada una de las variables presentes dentro del conjunto de datos.

Elementos a tener en cuenta:
- El significado y tipo (e.g. nominal/ordinal/intervalo/ratio) de cada una de las variables
- Identificación de valores faltantes y en caso de haberlos plantear las opciones para tratarlos
- Presencia de datos atípicos y en caso de haberlos identificar la manera de tratarlos

In [ ]:
df_train.info()

In [ ]:
df_train.columns

In [ ]:
df_test.columns

Formateamos los nombres de las variables

In [7]:
#Damos formato a las variables dejando todo en minuscula y sin espacios
def format_columns(df):
    result = df.copy()
    new_cols = []
    for col in result.columns:
        new_cols.append(re.sub(r'\s+', ' ',col.strip()).replace(' ','_').lower())
    result.columns = new_cols
    return result

*Descripciones de variables:* El significado y tipo (e.g. nominal/ordinal/intervalo/ratio) de cada una de las variables
Ejemplo:
- GRE_Score **(Ratio - Discreta)**: Puntuación en el Graduate Record Examination (GRE), un examen estandarizado utilizado para la admisión a programas de posgrado cuyo rango es desde 260 hasta 340.
- TOEFL_Score **(Ratio - Discreta)**: Puntuación en el Test of English as a Foreign Language (TOEFL), un examen estandarizado para medir la habilidad en inglés de hablantes no nativos cuyo rango es desde 0 hasta 120.
- University_Rating **(Ordinal)**: Calificación de la universidad, generalmente en una escala de 1 a 5, donde 5 es la más alta.

Falta completar---->

**agrupacion_diastolica:** Resultado Ultima Toma Tension Sistolica

**agrupacion_sistolica:** Resultado Ultima Toma Tension Diastolica

**cancer_mama_familiar:** Tienen Antecedentes De Cancer_Mama_Familiar

**cancer_otro_sitio:** El Usuario Tiene Cancer_Otro_Sitio Fuente Antecedentes

**cancer_otro_sitio_familiar:** Tienen Antecedentes De Cancer_Otro_Sitio_Familiar

**cant_fliar_cp:** Tiene Familiar con Cancer de Prostata?

**cant_fliar_riesgos:** Cantidad de riesgos del grupo Familiar

**cant_gr_flia:** Cantidad de personas en el grupo familiar

**cant_riesgos_flia_mean:** Cantidad promedio de riesgos en familia

**cantidad_serv_flia:** Cantidad de Servicios prestados a la familia en el ultimo semestre

**cantidad_servicios:** Cantidad de Servicios prestados al usuario en el ultimo semestre

**cerebral:** Ha presentado enfermedad cerebral?

**cerebral_familiar:** Tienen Antecedentes De Cerebral_Familiar

**conteo_dx_diferentes:** Cuantos Diagnosticos ha presentado en el ultimo semestre

**coronarios:** Ha presentado enfermedad coronaria?

**coronarios_familiar:** Tienen Antecedentes De Coronarios Familiar

**diabetes:** Ha presentado Diabetes?

**diabetes_familiar:** Tienen Antecedentes De Diabetes Familiar

**edad:** Edad del Usuario

**enfermedad_renal:** Ha presentado Enfermedad Renal?

**enfermedad_renal_familiar:** Tienen Antecedentes De Enfermedad Renal Familiar

**estado_civil:** Estado Civil del Usuario

**estrato:** Estrato sociodemografico del usuario

**genero:** Genero Usuario

**grupo_etareo:** Grupo Etareo

**hipertension:** Ha presentado HTA?

**hipertension_familiar:** Tienen Antecedentes De HTA Familiar

**imc:** Indice de masa corporal

**intercepto:** Intercepto del costo del ultimo semestre

**intercepto_flia:** Incercepto del costo familiar del ultimo semetre

**medicamentos:** Cantidad de medicamentos usados en el ultimo semestre

**medicina_especializada:** Cantidad de servicios de medicina especializada en el ultimo semestre

**medicina_general:** Cantidad de servicios de medicina general en el ultimo semestre

**min_tiempo_cp_fliar:** Tiempo desde la marca del CA de Prostata Familiar

**otros_antecedentes_vasculares:** Tienen Antecedentes De enfermedad vascular?

**parentesco:** Parentezco con el cotizante

**pendiente:** Pendiente del costo en el ultimo semestre

**pendiente_flia:** Pendiente del costo familiar en el ultimo semestre

**perdida_de_peso:** Peso perdido/ganado en el ultimo año

**programa:** Programa actual del usuario

**promedio_costo:** Promedio del costo en el ultimo semestre

**promedio_costo_flia:** Promedio del costo familiar en el ultimo semestre

**psa_max_gr_flia:** Resultado PSA Maximo del grupo familiar

**psa_min_gr_flia:** Resultado PSA Minimo del grupo familiar

**riesgos:** cantidad de riesgos del usuario

**target:** Resultado numerico del PSA

**target_clase:** Resultado anormal o normal del PSA

**tiempo_afiliacion:** Tiempo de afiliación del usuario

**tiempo_ultima_cita:** Tiempo desde la ultima cita del usuario

In [8]:
# Creamos un Transformer para renombrar columnas
class Rename_columns(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        result = X.copy()
        result = format_columns(result)
        result = result.rename(columns={'estado_civi':'estado_civil'}) #cambiamos el nombre a esta variable porque parece incompleto
        return result

In [9]:
# Preguntar si debo meter esto dentro del pipeline gigante de todo???
df_train_renamed = Rename_columns().fit_transform(df_train)
df_test_renamed = Rename_columns().fit_transform(df_test)

In [ ]:
df_train_renamed.columns

In [ ]:
df_test_renamed.columns

In [12]:
# Definir las columnas numéricas y categóricas
df_train_renamed_num = df_train_renamed.select_dtypes(include=['int64', 'float64'])
df_train_renamed_cat = df_train_renamed.select_dtypes(include=['object', 'category'])


In [ ]:
df_train_renamed_num.info()

In [ ]:
df_train_renamed_cat.info()

# *ojo*
Falta hacer análisis gráfico y estadisco de num y cat antes de imputar o después también



## tonterias para probar

In [ ]:
# Separar variable objetivo y características
X = df_train_renamed.drop('target', axis=1)
y = df_train_renamed['target']

# Separar en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocesamiento
# Definir las columnas numéricas y categóricas
numeric_features = df_train_renamed_num.select_dtypes(include=['int64', 'float64']).columns
categorical_features = df_train_renamed_cat.select_dtypes(include=['object', 'category']).columns

# Pipeline de preprocesamiento
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Función para optimización bayesiana para SVC + PCA
def optimize_svc_pca(C, gamma, n_components, kernel):
    """Función objetivo para optimización bayesiana del SVC + PCA"""
    model_svc = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('pca', PCA(n_components=int(n_components))),  # Número de componentes como parámetro
        ('classifier', SVC(probability=True, kernel=kernel, C=C, gamma=gamma))])
    
    # Validación cruzada y cálculo del AUC
    auc = cross_val_score(model_svc, X_train, y_train, cv=5, scoring='roc_auc').mean()
    return auc

# Lista de kernels a evaluar
kernels = ['linear', 'rbf', 'poly']

# Diccionario para almacenar los resultados de cada kernel
kernel_results = {}

# Loop para entrenar y optimizar SVC con diferentes kernels y número de componentes PCA
for kernel in kernels:
    print(f"Optimización bayesiana para el kernel: {kernel}")
    
    # Definir el espacio de búsqueda para C, gamma y n_components
    param_bounds = {
        'C': (0.1, 10),
        'gamma': (0.0001, 1) if kernel != 'linear' else (0.0001, 0.0001),  # gamma no afecta al kernel lineal
        'n_components': (2, X_train.shape[1])  # Número de componentes en PCA, rango: 2 a número total de features
    }
    
    # Optimización bayesiana para el kernel actual
    optimizer = BayesianOptimization(
        f=lambda C, gamma, n_components: optimize_svc_pca(C, gamma, n_components, kernel), 
        pbounds=param_bounds,
        random_state=42
    )
    
    optimizer.maximize(init_points=5, n_iter=25)
    
    # Guardar los mejores resultados
    kernel_results[kernel] = optimizer.max
    print(f'Mejores parámetros para kernel {kernel}: {optimizer.max["params"]}')
    print(f'Mejora del ROC-AUC: {optimizer.max["target"]}\n')

# Identificar el kernel con el mejor rendimiento
best_kernel = max(kernel_results, key=lambda k: kernel_results[k]['target'])
best_params = kernel_results[best_kernel]['params']

print(f'El mejor kernel es: {best_kernel} con ROC-AUC: {kernel_results[best_kernel]["target"]}')

# Entrenar el mejor modelo con el kernel ganador y los mejores parámetros
best_model_svc = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('pca', PCA(n_components=int(best_params['n_components']))),  # Usamos el mejor n_components
    ('classifier', SVC(probability=True, kernel=best_kernel, C=best_params['C'], gamma=best_params['gamma']))
])

best_model_svc.fit(X_train, y_train)
y_pred_proba = best_model_svc.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f'ROC-AUC en conjunto de prueba con el mejor kernel ({best_kernel}): {roc_auc}')

In [ ]:
"""
# otro prueba sin buscar o cambiar kernels

# Pipeline del modelo con optimización bayesiana
def train_svc(C, gamma):
    ### Función objetivo para optimización bayesiana ###
    model_svc = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('pca', PCA(n_components=10)),  # Opcional, si hay muchas características
        ('classifier', SVC(probability=True, kernel='rbf', C=C, gamma=gamma))])
    
    # Validación cruzada y cálculo del AUC
    auc = cross_val_score(model_svc, X_train, y_train, cv=5, scoring='roc_auc').mean()
    return auc

# Definir el espacio de búsqueda para C y gamma
param_bounds = {
    'C': (0.1, 10),
    'gamma': (0.0001, 1)
}

# Implementar la optimización bayesiana
optimizer = BayesianOptimization(f=train_svc, pbounds=param_bounds, random_state=42)
optimizer.maximize(init_points=5, n_iter=25)

# Imprimir el mejor resultado
print(f"Mejores parámetros con optimizacion bayesiana: {optimizer.max['params']}")
print(f"Mejor ROC-AUC con optimizxacion bayeasiana: {optimizer.max['target']}")

# Entrenar el mejor modelo encontrado
best_params = optimizer.max['params']
best_model_svc = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('pca', PCA(n_components=10)),
    ('classifier', SVC(probability=True, kernel='rbf', C=best_params['C'], gamma=best_params['gamma']))
])

best_model_svc.fit(X_train, y_train)
y_pred_proba = best_model_svc.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f'ROC_AUC en conjunto de prueba: {roc_auc}')

"""